# Azure AI Agents with Model Context Protocol (MCP) 지원 - Python

이 노트북은 Python에서 Model Context Protocol (MCP) 도구를 사용하여 Azure AI Agents를 활용하는 방법을 보여줍니다. 키리스 인증을 사용하여 외부 MCP 서버(예: Microsoft Learn)를 활용해 기능을 확장할 수 있는 지능형 에이전트를 생성하는 방법을 설명합니다.


## 필수 Python 패키지 설치

먼저 필요한 Python 패키지를 설치해야 합니다:
- **azure-ai-projects**: 핵심 Azure AI Projects SDK
- **azure-ai-agents**: 에이전트를 생성하고 관리하기 위한 Azure AI Agents SDK
- **azure-identity**: DefaultAzureCredential을 사용한 키리스 인증 제공
- **mcp**: Python용 Model Context Protocol 구현


## 키리스 인증의 이점

이 노트북은 **키리스 인증**을 시연하며, 다음과 같은 여러 가지 장점을 제공합니다:
- ✅ **API 키 관리 불필요** - Azure 기반의 ID 인증 사용
- ✅ **보안 강화** - 코드나 설정 파일에 비밀 정보를 저장하지 않음
- ✅ **자동 자격 증명 갱신** - Azure가 자격 증명 수명 주기를 관리
- ✅ **역할 기반 액세스 제어** - 세분화된 권한을 위해 Azure RBAC 사용
- ✅ **다중 환경 지원** - 개발 및 프로덕션 환경에서 원활하게 작동

`DefaultAzureCredential`은 사용 가능한 최적의 자격 증명 소스를 자동으로 선택합니다:
1. **Managed Identity** (Azure에서 실행 중일 때)
2. **Azure CLI** 자격 증명 (로컬 개발 중일 때)
3. **Visual Studio** 자격 증명
4. **환경 변수** (구성된 경우)
5. **인터랙티브 브라우저** 인증 (대체 옵션으로)


## 키리스 인증 설정

**키리스 인증을 위한 사전 준비 사항:**

### 로컬 개발 환경:
```bash
# Install Azure CLI and login
az login
# Verify your identity
az account show
```

### Azure 환경:
- Azure 리소스에서 **시스템 할당 관리 ID**를 활성화하세요.
- 관리 ID에 적절한 **RBAC 역할**을 할당하세요:
  - Azure OpenAI 액세스를 위한 `Cognitive Services OpenAI User`
  - Azure AI 프로젝트 액세스를 위한 `AI Developer`

### 환경 변수 (선택 사항):
```python
# These are automatically detected by DefaultAzureCredential
# AZURE_CLIENT_ID=<your-client-id>
# AZURE_CLIENT_SECRET=<your-client-secret>
# AZURE_TENANT_ID=<your-tenant-id>
```

**API 키나 연결 문자열이 필요하지 않습니다!** 🔐


In [ ]:
! pip install azure-ai-projects -U
! pip install azure-ai-agents==1.1.0b4 -U
! pip install azure-identity -U
! pip install mcp==1.11.0 -U

## 필수 라이브러리 가져오기

필요한 Python 모듈을 가져옵니다:
- **os, time**: 환경 변수와 지연 처리를 위한 표준 Python 라이브러리
- **AIProjectClient**: Azure AI Projects의 주요 클라이언트
- **DefaultAzureCredential**: Azure 서비스의 키리스 인증
- **MCP 관련 클래스**: MCP 도구 생성 및 관리, 승인 처리용


In [ ]:
import os, time
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.agents.models import McpTool, RequiredMcpToolCall, SubmitToolApprovalAction, ToolApproval


## MCP 서버 설정 구성

환경 변수를 사용하여 MCP 서버 구성을 설정하고 기본값을 제공합니다:
- **MCP_SERVER_URL**: MCP 서버의 URL (기본값은 Microsoft Learn API)
- **MCP_SERVER_LABEL**: MCP 서버를 식별하기 위한 라벨 (기본값은 "mslearn")

이 방법은 다양한 환경에서 유연한 구성을 가능하게 합니다.


In [ ]:
mcp_server_url = os.environ.get("MCP_SERVER_URL", "https://learn.microsoft.com/api/mcp")
mcp_server_label = os.environ.get("MCP_SERVER_LABEL", "mslearn")

## Azure AI 프로젝트 클라이언트 생성 (키리스 인증)

**키리스 인증**을 사용하여 Azure AI 프로젝트 클라이언트를 초기화합니다:
- **endpoint**: Azure AI Foundry 프로젝트 엔드포인트 URL
- **credential**: `DefaultAzureCredential()`을 사용하여 안전한 키리스 인증 제공
- **API 키 불필요**: 가장 적합한 인증 정보를 자동으로 탐지 및 사용

**인증 흐름:**
1. 관리 ID(Managed Identity)를 확인 (Azure 환경에서)
2. Azure CLI 자격 증명으로 대체 (로컬 개발 환경에서)
3. 필요에 따라 다른 사용 가능한 자격 증명 소스를 활용

이 접근 방식은 코드에서 API 키나 연결 문자열을 관리할 필요를 없애줍니다.


In [ ]:
project_client = AIProjectClient(
    endpoint="Your Azure AI Foundry Endpoint",
    credential=DefaultAzureCredential(),
)

## MCP 도구 정의 생성

Microsoft Learn MCP 서버에 연결하는 MCP 도구를 생성합니다:
- **server_label**: MCP 서버의 식별자
- **server_url**: MCP 서버의 URL 엔드포인트
- **allowed_tools**: 사용할 수 있는 도구를 제한하는 선택적 목록 (빈 목록은 모든 도구 사용 가능)

이 도구는 에이전트가 Microsoft Learn 문서와 리소스에 접근할 수 있도록 합니다.


In [ ]:
mcp_tool = McpTool(
    server_label=mcp_server_label,
    server_url=mcp_server_url,
    allowed_tools=[],  # Optional: specify allowed tools
)


## 에이전트 생성 및 대화 실행 (키리스 워크플로우)

이 포괄적인 섹션은 **키리스 에이전트 워크플로우**의 전체 과정을 보여줍니다:

1. **AI 에이전트 생성**: GPT-4.1 nano 모델과 MCP 도구를 사용하여 에이전트를 설정합니다.
2. **스레드 생성**: 소통을 위한 대화 스레드를 생성합니다.
3. **메시지 전송**: Azure OpenAI와 OpenAI의 차이에 대해 에이전트에게 질문합니다.
4. **도구 승인 처리**: 필요할 때 MCP 도구 호출을 자동으로 승인합니다.
5. **실행 모니터링**: 에이전트의 진행 상황을 추적하고 필요한 작업을 처리합니다.
6. **결과 표시**: 대화 내용과 도구 사용 세부 정보를 표시합니다.

**키리스 기능:**
- ✅ **하드코딩된 비밀 없음** - 모든 인증은 Azure ID로 처리됩니다.
- ✅ **기본적으로 안전함** - 역할 기반 액세스 제어를 사용합니다.
- ✅ **간소화된 배포** - 자격 증명 관리가 필요하지 않습니다.
- ✅ **감사 친화적** - 모든 액세스는 Azure ID를 통해 추적됩니다.

에이전트는 MCP 도구를 사용하여 Microsoft Learn 리소스에 접근하며, 완전한 보안과 API 키 관리 없이 작업을 수행합니다.


In [ ]:
with project_client:
    agents_client = project_client.agents

    # Create a new agent with keyless authentication
    # NOTE: To reuse existing agent, fetch it with get_agent(agent_id)
    agent = agents_client.create_agent(
        model="Your Azure OpenAI Model Deployment Name",
        name="my-mcp-agent",
        instructions="You are a helpful agent that can use MCP tools to assist users. Use the available MCP tools to answer questions and perform tasks.",
        tools=mcp_tool.definitions,
    )
    print(f"Created agent, ID: {agent.id}")
    print(f"MCP Server: {mcp_tool.server_label} at {mcp_tool.server_url}")

    # Create thread for communication
    thread = agents_client.threads.create()
    print(f"Created thread, ID: {thread.id}")

    # Create message to thread
    message = agents_client.messages.create(
        thread_id=thread.id,
        role="user",
        content="What's difference between Azure OpenAI and OpenAI?",
    )
    print(f"Created message, ID: {message.id}")

    # KEYLESS APPROACH: Handle tool approvals without hardcoded secrets
    
    # Option 1: Completely keyless (recommended for Azure identity-enabled MCP servers)
    # run = agents_client.runs.create(thread_id=thread.id, agent_id=agent.id, tool_resources=mcp_tool.resources)
    
    # Option 2: With minimal headers (if MCP server requires specific headers)
    # For demonstration purposes, using a placeholder header
    mcp_tool.update_headers("SuperSecret", "123456")  # Replace with actual auth if needed
    
    # Set approval mode - uncomment next line to disable approval requirement completely
    # mcp_tool.set_approval_mode("never")  # Fully automated, no approval needed
    
    run = agents_client.runs.create(thread_id=thread.id, agent_id=agent.id, tool_resources=mcp_tool.resources)
    print(f"Created run, ID: {run.id}")

    while run.status in ["queued", "in_progress", "requires_action"]:
        time.sleep(1)
        run = agents_client.runs.get(thread_id=thread.id, run_id=run.id)

        if run.status == "requires_action" and isinstance(run.required_action, SubmitToolApprovalAction):
            tool_calls = run.required_action.submit_tool_approval.tool_calls
            if not tool_calls:
                print("No tool calls provided - cancelling run")
                agents_client.runs.cancel(thread_id=thread.id, run_id=run.id)
                break

            tool_approvals = []
            for tool_call in tool_calls:
                if isinstance(tool_call, RequiredMcpToolCall):
                    try:
                        print(f"Approving tool call: {tool_call}")
                        
                        # KEYLESS APPROVAL OPTIONS:
                        
                        # Option 1: No headers (fully keyless)
                        # tool_approvals.append(
                        #     ToolApproval(
                        #         tool_call_id=tool_call.id,
                        #         approve=True,
                        #         headers={}  # No headers needed for keyless
                        #     )
                        # )
                        
                        # Option 2: With headers (if MCP server requires them)
                        tool_approvals.append(
                            ToolApproval(
                                tool_call_id=tool_call.id,
                                approve=True,
                                headers=mcp_tool.headers,  # Uses configured headers if needed
                            )
                        )
                    except Exception as e:
                        print(f"Error approving tool_call {tool_call.id}: {e}")

            print(f"tool_approvals: {tool_approvals}")
            if tool_approvals:
                agents_client.runs.submit_tool_outputs(
                    thread_id=thread.id, run_id=run.id, tool_approvals=tool_approvals
                )

        print(f"Current run status: {run.status}")

    print(f"Run completed with status: {run.status}")
    if run.status == "failed":
        print(f"Run failed: {run.last_error}")

    # Display run steps and tool calls
    run_steps = agents_client.run_steps.list(thread_id=thread.id, run_id=run.id)

    # Loop through each step
    for step in run_steps:
        print(f"Step {step['id']} status: {step['status']}")

        # Check if there are tool calls in the step details
        step_details = step.get("step_details", {})
        tool_calls = step_details.get("tool_calls", [])

        if tool_calls:
            print("  MCP Tool calls:")
            for call in tool_calls:
                print(f"    Tool Call ID: {call.get('id')}")
                print(f"    Type: {call.get('type')}")

        print()  # add an extra newline between steps

    # Fetch and log all messages
    messages = agents_client.messages.list(thread_id=thread.id)
    print("\nConversation:")
    print("-" * 50)
    for msg in messages:
        if msg.text_messages:
            last_text = msg.text_messages[-1]
            print(f"{msg.role.upper()}: {last_text.text.value}")
            print("-" * 50)

    # Example of dynamic tool management (keyless)
    print(f"\nDemonstrating keyless dynamic tool management:")
    print(f"Current allowed tools: {mcp_tool.allowed_tools}")
    print("✅ All operations completed using keyless authentication!")


---

**면책 조항**:  
이 문서는 AI 번역 서비스 [Co-op Translator](https://github.com/Azure/co-op-translator)를 사용하여 번역되었습니다. 정확성을 위해 최선을 다하고 있으나, 자동 번역에는 오류나 부정확성이 포함될 수 있습니다. 원본 문서의 원어 버전을 신뢰할 수 있는 권위 있는 자료로 간주해야 합니다. 중요한 정보의 경우, 전문적인 인간 번역을 권장합니다. 이 번역 사용으로 인해 발생하는 오해나 잘못된 해석에 대해 책임을 지지 않습니다.
